<a href="https://colab.research.google.com/github/jhleee/langgraph-exercise/blob/master/LangGraph_exercise2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install langgraph langchain-google-genai python-dotenv


In [19]:
# gemini 기본 셋팅

import os
from google.colab import userdata

api_key = userdata.get('GOOGLE_API_KEY')


from langchain_google_genai import ChatGoogleGenerativeAI
# 모델 로드
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_output_tokens=200,
    google_api_key=api_key,
)
llm.invoke("Hello")

AIMessage(content='Hello there! How can I help you today?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-b408f638-9f1f-4126-a01e-b9059e239e86-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

In [57]:
import re

# LLM을 사용한 함수들을 작성

def ask_something(text):
  prompt = f"""당신은 프로그래밍 전문가입니다.
  다음 유저의 질문에 대해 다음 지시에 따라 CoT를 사용하여 따라 답변하시오.
  1. 사용자의 질문이 위법행위를 내포하는 경우 답변을 중지하고, 명령어 <END/> 라고 답변하고 대화를 종료하시오.
  2. 사용자의 질문이 모호한 경우 되묻는 명령어 <ASK/> 라고 답변하고 대화를 종료하시오.
  3. (CoT) 질문의 의도 파악을  <ANALYZE> 태그를 사용하시오.
  4. (CoT) 관련 사실을 확인하고 <VERIFY> 태그를 사용하시오.
  5. (CoT) 최종적인 답변을 구조화 하고 <RESPONSE> 태그를 사용하시오.

  IMPORTANT: 컨텐트가 있는 태그의 경우, 열고 닫음이 필수입니다.

  사용자의 질문:
  {text}
  """
  return llm.invoke(prompt)

def structrure(response):
  content = response.content
  anl = re.search(r'<ANALYZE>(.*?)</ANALYZE>', content)
  ver = re.search(r'<VERIFY>(.*?)</VERIFY>', content)
  res = re.search(r'<RESPONSE>(.*?)</RESPONSE>', content)

  return {
      "analyze": anl.group(1),
      "verify": ver.group(1),
      "answer": res.group(1),
  }

def finish(content):
  return f"AI Agent: \n* 분석:{content['analyze']}\n* 사실관계:{content['verify']}\n* 답변:{content['answer']}"


In [58]:
# ASK -> FIN 의 간단한 그래프를 구성한다.

from langgraph.graph import Graph

workflow = Graph()
workflow.add_node("ASK", ask_something)
workflow.add_node("STRUCT", structrure)
workflow.add_node("FIN", finish)

workflow.add_edge("ASK", "STRUCT")
workflow.add_edge("STRUCT", "FIN")

workflow.set_entry_point("ASK")
workflow.set_finish_point("FIN")

# 마지막엔 컴파일 !

app = workflow.compile()

In [59]:
for output in app.stream("서울의 수도는?"):
  # key:value의 dict를 yield 해준다.
  print("* Strem ...")
  for k, v in output.items():
    print(f"NODE[{k}]: {v}")

* Strem ...
NODE[ASK]: content='<ANALYZE> 사용자는 서울의 수도를 묻고 있습니다.  이는 지리적 위치에 대한 기본적인 질문입니다. </ANALYZE>\n\n<VERIFY> 서울은 대한민국의 수도입니다.  이는 일반적으로 알려진 사실이며, 다양한 지리 정보 사이트 및 백과사전에서 확인 가능합니다. </VERIFY>\n\n<RESPONSE> 서울은 대한민국의 수도입니다. </RESPONSE>\n' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-967a852b-27be-4145-9f0d-f0984d8b66e5-0' usage_metadata={'input_tokens': 251, 'output_tokens': 98, 'total_tokens': 349, 'input_token_details': {'cache_read': 0}}
* Strem ...
NODE[STRUCT]: {'analyze': ' 사용자는 서울의 수도를 묻고 있습니다.  이는 지리적 위치에 대한 기본적인 질문입니다. ', 'verify': ' 서울은 대한민국의 수도입니다.  이는 일반적으로 알려진 사실이며, 다양한 지리 정보 사이트 및 백과사전에서 확인 가능합니다. ', 'answer': ' 서울은 대한민국의 수도입니다. '}
* Strem ...
NODE[FIN]: AI Agent: 
* 분석: 사용자는 서울의 수도를 묻고 있습니다.  이는 지리적 위치에 대한 기본적인 질문입니다. 
* 사실관계: 서울은 대한민국의 수도입니다.  이는 일반적으로 알려진 사실이며, 다양한 지리 정보 사이트 및 백과사전에서 확인 가능합니다. 
* 답변: 서울은 대한민국의 수도입니다. 
